In [12]:
%%capture

!pip3 install pandas
!pip3 install numpy

import pandas as pd 
import numpy as np

In [13]:
#contains movie id and corresponding imdbId and tmdbId
links = pd.read_csv('links.csv')

# contains movieId title and genres that the movie falls under
movies = pd.read_csv('movies.csv')

# contains ratings for movieId by users with a timestampe(unsure what this is)
ratings = pd.read_csv('ratings.csv')

#contains UserId associated with a movieId with the tag the user rated with and a time stamp
tags = pd.read_csv('tags.csv')

In [14]:
#drops userId and timestamp not needed
ratings.drop(['userId','timestamp'], axis = 1, inplace = True)
#adds a column for mean and count for the rating 
ratings = ratings.groupby('movieId').agg(['mean','count'])
#renames columns
ratings.columns = ['Average_Rating', 'Total_Rating']

# merges ratings with links csv by movieId
movies = movies.merge(ratings, how = 'inner', on = 'movieId')
movies = movies.merge(links,  how ='inner', on = 'movieId')

In [15]:
#creates a dictionary for title and year 
moviesByYear = []
for x in range(len(movies)):
    moviesByYear.append([
        int(movies.movieId[x]), 
        str(movies.title[x][0:movies.title[x].rfind('(')]),
        str(movies.title[x][movies.title[x].rfind('(')+1:movies.title[x].rfind(')')])
    ])

In [16]:
#replaces current title with new title and year
moviesByYearDf = pd.DataFrame(moviesByYear, columns=['movieId',"title","year"])
movies.drop(columns='title', inplace = True)
movies = moviesByYearDf.merge(movies, how = 'inner', on = 'movieId')
movies = movies.round({'Average_Rating': 4})

,movieId,title,year,genres,Average_Rating,Total_Rating,imdbId,tmdbId
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,3.8937,57309,114709,862.0
1,2,Jumanji,1995,Adventure|Children|Fantasy,3.2515,24228,113497,8844.0
2,3,Grumpier Old Men,1995,Comedy|Romance,3.1420,11804,113228,15602.0
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance,2.8535,2523,114885,31357.0
4,5,Father of the Bride Part II,1995,Comedy,3.0584,11714,113041,11862.0
...,...,...,...,...,...,...,...,...
59042,209157,We,2018,Drama,1.5000,1,6671244,499546.0
59043,209159,Window of the Soul,2001,Documentary,3.0000,1,297986,63407.0
59044,209163,Bad Poems,2018,Comedy|Drama,4.5000,1,6755366,553036.0
59045,209169,A Girl Thing,2001,(no genres listed),3.0000,1,249603,162892.0


In [23]:
movies.to_csv('DataforMovies.csv')

In [17]:

print(movies.head())

   movieId                         title  year  \
0        1                    Toy Story   1995   
1        2                      Jumanji   1995   
2        3             Grumpier Old Men   1995   
3        4            Waiting to Exhale   1995   
4        5  Father of the Bride Part II   1995   

                                        genres  Average_Rating  Total_Rating  \
0  Adventure|Animation|Children|Comedy|Fantasy        3.893708         57309   
1                   Adventure|Children|Fantasy        3.251527         24228   
2                               Comedy|Romance        3.142028         11804   
3                         Comedy|Drama|Romance        2.853547          2523   
4                                       Comedy        3.058434         11714   

   imdbId   tmdbId  
0  114709    862.0  
1  113497   8844.0  
2  113228  15602.0  
3  114885  31357.0  
4  113041  11862.0  


In [ ]:
#Apparenty nothing needs to be done with this because genres are already added by relevance

#Contains movieid tag id and relevance
genome_scores = pd.read_csv('genome-scores.csv')

#contains tagId and tag looks like genres
genome_tags = pd.read_csv('genome-tags.csv')


#keeping this in case filteration is needed
genome_scores['Genre'] = genome_scores['tagId'].map(genome_tags.set_index('tagId')['tag'])
genome_scores = genome_scores[genome_scores['relevance'] > .75]
genome_scores.reset_index(drop=True, inplace=True)